In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm.notebook import tqdm

In [2]:
PRED_PATH = "../hota/2020_12_16_Montana_at_Washington.txt"
GT_PATH = "data\processed\forTest\images\2018-11-28_Virginia_at_Maryland"

In [3]:
def xyxy_to_xywh(bbox):
    x1, y1, x2, y2 = bbox
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    width = np.abs(x2 - x1)
    height = np.abs(y2 - y1)
    
    return [x_center, y_center, width, height]

In [4]:
def create_gt_file(folder='2020_12_16_Montana_at_Washington'):
 
    image_folder = '..\\data\\ncaa_dataset\\images'

    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    images.sort()

    num = 0

    with open(f'..\\data\\ncaa_dataset\\gt_xywh.txt', 'w') as output_file:

        for i, img in enumerate(tqdm(images), start=1):
            img_path = os.path.join(image_folder, img)
            anno_path = img_path.replace('jpg', 'json')
            anno_path = anno_path.replace('images', 'annos')

            if not os.path.isfile(anno_path):
                continue

            with open(anno_path, 'r') as f:
                data = json.load(f)


            for shape in data['shapes']:

                label = shape['label']

                if label.isdigit():

                    x1, y1 = shape['points'][0]
                    x2, y2 = shape['points'][1]

                    width = np.abs(x2 - x1)
                    height = np.abs(y2 - y1)

                    # bbox_xywh = ops.xyxy2xywh(np.array([x1, y1, x2, y2], dtype=float))    

                    line = f"{i} {label} {x1:.0f} {y1:.0f} {width:.0f} {height:.0f} -1 -1 -1 0\n"

                    output_file.write(line)

In [5]:
create_gt_file()

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'data\\ncaa_dataset\\images'

In [ ]:
def xywh2xyxy(x: pd.DataFrame):
    """
    Convert bounding box coordinates from (x, y, width, height) format to (x1, y1, x2, y2) format where (x1, y1) is the
    top-left corner and (x2, y2) is the bottom-right corner.

    Args:
        x (np.ndarray) or (torch.Tensor): The input bounding box coordinates in (x, y, width, height) format.
    Returns:
        y (np.ndarray) or (torch.Tensor): The bounding box coordinates in (x1, y1, x2, y2) format.
    """
    y = np.copy(x)
    y[..., 2] = (x.iloc[:, 2])  # x center
    y[..., 3] = (x.iloc[:, 3]) # y center
    y[..., 4] = x.iloc[:, 4] - x.iloc[:, 2]  # width
    y[..., 5] = x.iloc[:, 5] - x.iloc[:, 3]  # height
    return y

In [ ]:
data = pd.read_table('../data/anno_test_track/2020_12_16_Montana_at_Washington_gt_xyxy.txt', 
                    delimiter=' ', header=None)
print(data.head())
data = xywh2xyxy(data)
df_data = pd.DataFrame(data)
df_data.head()

df_data.to_csv(r'../data/anno_test_track/2020_12_16_Montana_at_Washington_gt_xywh.txt', 
                header=None, index=None, sep=' ', mode='a') # type: ignore